# **DEPENDENCIES**

In [6]:
import numpy as np
import cv2

import os

In [ ]:
from google.colab import drive; drive.mount("/content/drive")

# **DATASET**

In [7]:
faces_dataset_path = "/content/drive/MyDrive/data_assn2_cv/actors/faces"
